<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Data_Engineering_Essentials(news%20sentiment%20analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Fetch Data from an API](https://requests.readthedocs.io/en/latest/)**

In [ ]:
!pip install requests

In [ ]:
import requests

In [ ]:
payload = {"dx_hcc": "dx", "dos_year": "v24_2022", "drf": "CNA", "search": "I50814", "start": 0}

In [ ]:
response = requests.post(url = 'https://www.hccreference.com/api/search',headers={"content-type":"application/json"},json =payload)

In [ ]:
response.text

In [ ]:
response.status_code

# **News Sentiment Analysis**

https://newsapi.org/

https://signup.snowflake.com/

In [ ]:
!pip install snowflake-connector-python

In [ ]:
#data extraction & transformation
import json
import requests
import datetime
from datetime import date

# Your News API key
api_key = ''

end_day = date.today()
start_day = end_day - datetime.timedelta(days=3)

def extract_news_api():
    url_extractor = f"https://newsapi.org/v2/everything?q='USA Election'&from={start_day}&to={end_day}&sortBy=popularity&apiKey={api_key}&language=en"
    print("The URL to extract the data is: ", url_extractor)
    response = requests.get(url_extractor)
    data = response.json()

    if data['status'] == 'ok':
        news_data = []
        for article in data['articles']:
            news_title = article['title']
            timestamp = article['publishedAt']
            url_source = article['url']
            partial_content = article['content'] or ""

            if len(partial_content) >= 200:
                partial_content = partial_content[:199]

            if '.' in partial_content:
                trimmed_part = partial_content[:partial_content.rindex('.')]
            else:
                trimmed_part = partial_content

            news_data.append((news_title, timestamp, url_source, trimmed_part))

        return news_data

In [ ]:
news_data=extract_news_api()

In [ ]:
news_data

In [ ]:
#loading data in Snowflake
import snowflake.connector

# Replace with your actual Snowflake credentials
SNOWFLAKE_ACCOUNT = ''
SNOWFLAKE_USER = ''
SNOWFLAKE_PASSWORD = ''
SNOWFLAKE_DATABASE = 'data_engineering'
SNOWFLAKE_SCHEMA = 'PUBLIC'
SNOWFLAKE_TABLE = 'NEWS_DATA'
SNOWFLAKE_WAREHOUSE = 'COMPUTE_WH'

def load_data_to_snowflake(data):
    conn = snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA
    )

    cursor = conn.cursor()

    # Assuming the table has columns: TITLE, TIMESTAMP, URL, CONTENT
    insert_query = f"""
    INSERT INTO {SNOWFLAKE_TABLE} (TITLE, TIMESTAMP, URL, CONTENT)
    VALUES (%s, %s, %s, %s)
    """

    try:
        cursor.executemany(insert_query, data)
        conn.commit()
        print(f"{len(data)} rows inserted successfully.")
    except Exception as e:
        print("Error inserting data:", e)
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    news_data = extract_news_api()
    if news_data:
        load_data_to_snowflake(news_data)


# **SQL Queries**

In [ ]:
DROP DATABASE IF EXISTS data_engineering;

CREATE DATABASE data_engineering;

use data_engineering;

CREATE or replace TABLE NEWS_DATA (TITLE TEXT, TIMESTAMP TEXT, URL TEXT, CONTENT TEXT);

select * from NEWS_DATA;

SELECT
    URL,
    title,
    SNOWFLAKE.CORTEX.COMPLETE(
        'mistral-large2',
        'Classify the sentiment of the following text as Positive, Negative, or Neutral: ' || title || '. Return only the sentiment.'
    ) as content_sentiment,
    CONTENT,
    SNOWFLAKE.CORTEX.COMPLETE(
        'mistral-large2',
        'Classify the sentiment of the following text as Positive, Negative, or Neutral: ' || CONTENT || '. Return only the sentiment.'
    ) as content_sentiment
FROM NEWS_DATA;


--summarization
select SNOWFLAKE.CORTEX.COMPLETE(
        'mistral-large2',
        'Human: You are a summarisation assistant. Your task is to summarise product reviews given to you as a list. Within this list, there are individual product reviews in an array.
    Create a JSON document with the following fields:
    summary - A summary of these reviews in less than 200 words
    overall_sentiment - The overall sentiment of the reviews
    sentiment_confidence - How confident you are about the sentiment of the reviews
    reviews_positive - The percent of positive reviews
    reviews_neutral - The percent of neutral reviews
    reviews_negative - The percent of negative reviews
    action_items - A list of action items to resolve the customer complaints (don''t put something which is already good and there is no customer complaint)
    Your output should be raw JSON - do not include any sentences or additional text outside of the JSON object.
    Assistant:'||
    'I recently had a very disappointing and stressful experience on my flight from Delhi (DEL) to Vancouver (YVR). The whole journey was plagued with issues and it left me with a bitter taste in my mouth.

To begin with, the flight schedule kept changing multiple times before my departure date. This caused a lot of inconvenience as I had to keep rearranging my plans and make last-minute changes. The lack of proper communication about these changes only added to my frustration.

On the day of my flight, I arrived at the airport to find out that my flight was delayed by four hours. This was not only frustrating but also exhausting as I had a connecting flight to catch in Vancouver. Despite the long delay, there was no proper explanation or updates provided by the airline staff.

When I finally boarded the flight, I was shocked to see the condition of the aircraft. The seats were old and uncomfortable, and there was a musty smell in the cabin. Additionally, the in-flight entertainment system was not working, which made the long journey even more unbearable.

The worst part of the flight was the food. It was tasteless and poorly prepared. To make matters worse, the flight attendants were rude and unaccommodating when I requested a vegetarian meal, which I had pre-booked. I had to settle for some fruits and crackers for my entire meal.

To top it all off, my baggage did not arrive with me at YVR. It was frustrating to have to wait for hours at baggage claim only to find out that my luggage was still in Delhi. I had to file a report and wait for two days before my baggage was finally delivered to my hotel.

Overall, my experience with this flight was highly disappointing. The constant delays, poor service, uncomfortable seats, and lost baggage left me stressed and exhausted. I expected much better from this airline, especially for the price I paid for my ticket. I would not recommend this airline to anyone and I will definitely think twice before booking another flight with them.'
    ) ;

--bit cleanup
select parse_json(replace(replace(SNOWFLAKE.CORTEX.COMPLETE(
'mistral-large2',
'Human: You are a summarisation assistant. Your task is to summarise product reviews given to you as a list. Within this list, there are individual product reviews in an array.
Create a JSON document with the following fields:
summary - A summary of these reviews in less than 200 words
overall_sentiment - The overall sentiment of the reviews
sentiment_confidence - How confident you are about the sentiment of the reviews
reviews_positive - The percent of positive reviews
reviews_neutral - The percent of neutral reviews
reviews_negative - The percent of negative reviews
action_items - A list of action items to resolve the customer complaints (don''t put something which is already good and there is no customer complaint)
Your output should be raw JSON - do not include any sentences or additional text outside of the JSON object.
Assistant:'||
'I recently had a very disappointing and stressful experience on my flight from Delhi (DEL) to Vancouver (YVR). The whole journey was plagued with issues and it left me with a bitter taste in my mouth.

To begin with, the flight schedule kept changing multiple times before my departure date. This caused a lot of inconvenience as I had to keep rearranging my plans and make last-minute changes. The lack of proper communication about these changes only added to my frustration.

On the day of my flight, I arrived at the airport to find out that my flight was delayed by four hours. This was not only frustrating but also exhausting as I had a connecting flight to catch in Vancouver. Despite the long delay, there was no proper explanation or updates provided by the airline staff.

When I finally boarded the flight, I was shocked to see the condition of the aircraft. The seats were old and uncomfortable, and there was a musty smell in the cabin. Additionally, the in-flight entertainment system was not working, which made the long journey even more unbearable.

The worst part of the flight was the food. It was tasteless and poorly prepared. To make matters worse, the flight attendants were rude and unaccommodating when I requested a vegetarian meal, which I had pre-booked. I had to settle for some fruits and crackers for my entire meal.

To top it all off, my baggage did not arrive with me at YVR. It was frustrating to have to wait for hours at baggage claim only to find out that my luggage was still in Delhi. I had to file a report and wait for two days before my baggage was finally delivered to my hotel.

Overall, my experience with this flight was highly disappointing. The constant delays, poor service, uncomfortable seats, and lost baggage left me stressed and exhausted. I expected much better from this airline, especially for the price I paid for my ticket. I would not recommend this airline to anyone and I will definitely think twice before booking another flight with them.'
),'```',''),'json','')) ;

--entity extraction example
select parse_json(replace(replace(SNOWFLAKE.CORTEX.COMPLETE(
'mistral-large2','You are a helpful assistant. Please extract the following details from the extracted_text & format the output as JSON using the keys

<details>
doctor_name: The doctor or provider''s full name
provider_id: The doctor or provider''s ID
patient_name: The patient''s full name
patient_id: The patient''s ID
patient_gender: The patient''s gender
patient_age: The patient''s age
admitted_date: Date the patient was admitted to the hospital
discharge_date: Date the patient was discharged from the hospital
discharged_to: where the patient was released or discharged to
drug_allergies: The patient''s known drug allergies (NKDA)
patient_medical_history: The patient''s medical history (PMHx)
family_medical_history: The patient''s family medical history (FHx)
social_habits: The patient''s social habits (Social HX)
</details>

<keys>
doctor_name
provider_id
patient_name
patient_id
patient_gender
patient_age
admitted_date
discharge_date
discharged_to
drug_allergies
patient_medical_history
family_medical_history
social_habits
</keys>

<extracted_text>'||
'Not a Memorial Hospital
Of Collier
Reg: PN/S/11011, Non-Profit
Contact: (999)-(888)-(1234)

Physician Hospital Discharge Summary

Provider: Mateo Jackson, Phd

Patient: John Doe

Provider''s Pt ID: 00988277891

Patient Gender: Male

Attachment Control Number: XA/7B/00338763

Visit (Encounter)

Admitted: 07-Sep-2020

Discharged: 08-Sep-2020

Discharged to: Home with support services

Assessment

Reported Symptoms / History
of present illness:

35 yo M c/o stomach problems since 2 montsh ago. Patient
reports epigastric abdominal pain non-radiating. Pain is
described as gnawing and burning, intermitent lasting 1-2
hours, and gotten progressively worse. Antacids used to
alleviate pain but not anymore; nothing exhacerbates pain.
Pain unrelated to daytime or to meals. Patient denies
constipation or diarrhea. Patient denies blood in stool but
have noticed them darker. Patient also reports nausea.
Denies recent illness or fever. He also reports fatigue
since 2 weeks ago and bloating after eating.
Patient ID: NARH-36640
ROS: Negative except for above findings
Meds: Motrin once/week. Tums previously.
PMHX: Back pain and muscle spasms. No HX of surgery. NKDA.
FHX: Uncle has a bleeding ulcer.

Discharge

Discharge Studies Summary:
Some activity restrictions suggested, full course of
antibiotics, check back with physican in case of relapse,
strict diet

Social Hx: Smokes since 15 yo, 1/2-1 PPD. No recent EtOH
use. Denies illicit drug use. works on high elevation
construction. Fast food diet. Exercises 3-4 times/week but
stopped 2 weeks ago.

VS/3S/Q990-7550/1090001004290'||'</extracted_text>'),'```',''),'json',''));




--nlp
select SNOWFLAKE.CORTEX.SENTIMENT('I am just so frustrated I don''t know how the story ends!');

--nlu
select snowflake.cortex.complete('mistral-large','Would the user like next game or next book in the series based on the review, strictly return yes or no without any additional text: ' ||'I am just so frustrated I don’t know how the story ends!');
